# Segmentation trial acquisitions (calcium imaging)
* Segmentation of (motion corrected) images


In [4]:
import numpy as np
from cellpose import models
import napari
import os
from scripts.sample_db import SampleDB
from tifffile import imwrite, imread

# Load the sample database
db_path = r'\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\sample_db.csv'
sample_db = SampleDB()
sample_db.load(db_path)
print(sample_db)

# Loading experiment
sample_id = '20220426_RM0008_130hpf_fP1_f3'
exp = sample_db.get_sample(sample_id)
print(exp)

# Import model
model_path = r'D:\montruth\cellpose\models\CP_20230803_101131' 
model = models.CellposeModel(model_type=model_path, gpu=True)

# Making shortcuts of sample parameters/information
sample = exp.sample
raw_path = exp.paths["raw_path"]
anatomy_path = exp.paths["anatomy_path"]
em_path = exp.paths["em_path"]
n_planes = exp.params_lm.n_planes
n_frames = exp.params_lm.n_frames
n_slices = exp.params_lm.lm_stack_range
doubling = 2 if exp.params_lm.doubling else 1

# Getting paths of the trial acquisitions
trial_paths = os.listdir(os.path.join(raw_path, "trials"))

# Define the path for the preprocessed folder
preprocessed_folder = os.path.join(r"\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\2P_RawData\2022-04-26\f3", "preprocessed")
os.makedirs(preprocessed_folder, exist_ok=True)

# Define the path for the masks folder
masks_folder = os.path.join(preprocessed_folder, "..", "masks")
os.makedirs(masks_folder, exist_ok=True)

SampleDB(sample_ids=['20220426_RM0008_130hpf_fP1_f3'])
sample=Sample(id='20220426_RM0008_130hpf_fP1_f3', parents_id='P1', genotype='aTubulin:GCamp6s x GAD1b:DsRed', phenotype='positive expression, nacre', dof='21.04.2022-09:30', hpf=130, body_length_mm=4) params_odor=ParamsOdor(odor_list=['Ala', 'Ser', 'Ctrl', 'TCA', 'GCA', 'TDCA', 'Cad', 'SA'], odor_sequence=['Ala', 'Ser', 'Ctrl', 'TCA', 'GCA', 'TDCA', 'Cad', 'SA', 'Cad', 'TDCA', 'GCA', 'TCA', 'Ser', 'Ctrl', 'Ala', 'SA', 'Ctrl', 'Cad', 'Ala', 'Ser', 'TDCA', 'GCA', 'TCA', 'SA'], odor_concentration_uM=[OdorConcentration(name='Ala', concentration_mM=100.0), OdorConcentration(name='Ser', concentration_mM=100.0), OdorConcentration(name='Ctrl', concentration_mM=100.0), OdorConcentration(name='TCA', concentration_mM=10.0), OdorConcentration(name='GCA', concentration_mM=10.0), OdorConcentration(name='TDCA', concentration_mM=10.0), OdorConcentration(name='Cad', concentration_mM=300.0), OdorConcentration(name='SA', concentration_mM=0.0)], n_tri

In [18]:
%%time
# Segment all trials per plane
all_images = []
all_masks = []


# Loop through each plane and process images
for plane in range(n_planes*doubling):
    plane_str = f"{plane:02d}"
    images = []
    for trial_path in trial_paths:
        image_path = os.path.join(preprocessed_folder, f"rigid_corrected_plane_{plane:02d}_{trial_path}")
        images.append(imread(image_path))
    images = np.array(images)
    all_images.append(images)

    # Segment the images using Cellpose
    masks, flows, styles = model.eval(images, channels=[0, 0], resample=False, cellprob_threshold=-3, flow_threshold=0, augment=False, stitch_threshold=0.01)
    
    # Save the initial masks for each plane
    mask_plane_path = os.path.join(masks_folder, f"mask_rigid_corrected_plane_{plane_str}.tif")
    imwrite(mask_plane_path, masks)
    print(f"Mask for plane {plane} saved at {mask_plane_path}")

    # Store the masks for visualization
    all_masks.append(masks)

# Convert lists to numpy arrays for visualization
all_masks = np.array(all_masks)


  0%|          | 0/23 [00:00<?, ?it/s]D:\montruth\PycharmProjects\samplePy\.venv\lib\site-packages\cellpose\metrics.py:196: RuntimeWarning: invalid value encountered in divide
  iou = overlap / (n_pixels_pred + n_pixels_true - overlap)
100%|██████████| 23/23 [00:00<00:00, 132.95it/s]


Mask for plane 0 saved at \\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\2P_RawData\2022-04-26\f3\preprocessed\..\masks\mask_rigid_corrected_plane_00.tif


100%|██████████| 23/23 [00:00<00:00, 148.38it/s]


Mask for plane 1 saved at \\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\2P_RawData\2022-04-26\f3\preprocessed\..\masks\mask_rigid_corrected_plane_01.tif


100%|██████████| 23/23 [00:00<00:00, 119.78it/s]


...

Mask for plane 5 saved at \\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\2P_RawData\2022-04-26\f3\preprocessed\..\masks\mask_rigid_corrected_plane_05.tif


100%|██████████| 23/23 [00:00<00:00, 135.28it/s]


Mask for plane 6 saved at \\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\2P_RawData\2022-04-26\f3\preprocessed\..\masks\mask_rigid_corrected_plane_06.tif


100%|██████████| 23/23 [00:00<00:00, 182.52it/s]


Mask for plane 7 saved at \\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\2P_RawData\2022-04-26\f3\preprocessed\..\masks\mask_rigid_corrected_plane_07.tif
CPU times: total: 57min 28s
Wall time: 11min 45s


In [19]:
np.array(all_images).shape

(8, 24, 256, 512)

In [20]:
# Visualize masks

viewer = napari.Viewer()
viewer.add_image(np.array(all_images), name='images_rigid_corrected')
viewer.add_labels(all_masks, name='masks')

<Labels layer 'masks' at 0x2d1085f6770>

In [ ]:
# Check which rois are in all trials 

# Lists to store masks for all planes
all_masks = []
all_trials_masks = []

# Loop through each plane and process images
for plane in range(1):
    plane_str = f"{plane:02d}"
    print(plane_str)

    # Load masks of plane
    ...

    # Dictionary to store the presence of each label across all trials
    label_presence = {}

    # Track the presence of each label in all trials
    for trial in range(len(images)):
        print(trial)
        unique_labels = np.unique(masks[trial])
        for label in unique_labels:
            if label != 0:  # Exclude background
                if label not in label_presence:
                    label_presence[label] = set()
                label_presence[label].add(trial)

    # Find labels present in all trials
    labels_in_all_trials = [label for label, trials in label_presence.items() if len(trials) == len(images)]

    # Calculate the percentage of segmented cells present in all trials
    total_labels = len(label_presence)
    print(total_labels)
    percentage_in_all_trials = (len(labels_in_all_trials) / total_labels) * 100
    print(f"Percentage of segmented cells present in all trials for plane {plane}: {percentage_in_all_trials:.2f}%")

    # Create a new mask for cells present in all trials
    all_trials_mask = np.zeros_like(masks)
    for label in labels_in_all_trials:
        all_trials_mask[masks == label] = label

    # Store the masks for visualization
    all_masks.append(masks)
    all_trials_masks.append(all_trials_mask)

# Convert lists to numpy arrays for visualization
all_masks = np.array(all_masks)
all_trials_masks = np.array(all_trials_masks)

# Visualize the results in Napari
viewer = napari.Viewer()
for plane in range(exp.params_lm.n_planes):
    viewer.add_image(images, name=f'Original Images Plane {plane}')
    viewer.add_labels(all_masks[plane], name=f'Segmented Masks Plane {plane}')
    viewer.add_labels(all_trials_masks[plane], name=f'Cells Present in All Trials Plane {plane}')